
<img src="img/logo_wiwi.png" width="25%" align="left">

<img src="img/decision_analytics_logo.png" width="17%" align="right">



<br><br><br><br><br><br><br><br>



# Algorithmen und Datenstrukturen(A+D)-Projekt 

**Sommersemester 2023**


# 7. Experimente

<br>

<br>
<br>

**Michael Römer, Till Porrmann**

Juniorprofessur für Decision Analytics  | Universität Bielefeld

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numba import njit

## Was ist das Thema dieser Materialien?

**Wir betrachten ein paar hilfreiche Techniken für das Durchführen und Auswerten von Experimenten mit Algorithmen**

- Aufrufen mehrerer Instanzen und Algorithmen
- Datenstrukturen für Ergebnisse
- Aufbereiten der Ergebnisse mit `data frames`
- Speichern von Ergebnissen in Dateien
- Auslesen von Ergebnissen aus Dateien


**Bitte beachten Sie:**

Diese Materialien sollen Ihnen anhand von Beispielen einige Techniken zeigen, wie man Experimente durchführen kann und Ergebnisse sammel und aufbereiten kann.

Sie können natürlich in Ihrem Projekt auch anders vorgehen.




# Rückblick: Algorithmen für das TSP

### Beachte: 
Wir haben hier alle Implementierungen der Algorithmen so geändert, dass `distance_matrix` das erste Argument ist

## Das TSP als Beispiel

Für das TSP haben wir 
- für viele Instanzen optimale Zielfunktionswerte: http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html
- das Python-Paket `python-tsp`, mit dem wir unsere Algorithmen vergleichen können (https://github.com/fillipe-gsm/python-tsp)
- einige Algorithmen implementiert
  - Nearest Neighbor
  - Multistart Nearest Neighbor
  - Informierter Greedy mit Rollout als heuristischer Funktion
  - Beam Search (mit unterschiedlichen Breiten)
  - Informiertes Beam Search (mit unterschiedlichen Breiten und limitierter Anwendung der Heuristik)

**Wie können wir**
- sinnvoll mit mehreren (z.B. > 10) Instanzen experimentieren?
- die Ergebnisse in Dateien speichern und aufbereiten?

# Code mit unseren bisherigen Algorithmen / Varianten

In [2]:
from python_tsp.distances import tsplib_distance_matrix

#tsplib_file = "./../problems/tsp/instances/gr48.tsp" # optimale Lösung 5046 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
#tsplib_file = "./../problems/tsp/instances/brazil58.tsp" # optimale Lösung 25395 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
tsplib_file = "./../problems/tsp/instances/berlin52.tsp" # optimale Lösung 7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)

distance_matrix = tsplib_distance_matrix(tsplib_file)

## Eine Routine zum Evaluieren einer Lösung

- immer wenn man nicht-triviale Algorithmen für Optimierungsprobleme entwickelt, sollte man einen "Solution-Checker" schreiben / nutzen

In [3]:
def evaluate_tsp_solution(distance_matrix, permutation):
    n = len(distance_matrix)
    if len(permutation) != n:
        print ("Wrong number of nodes")
        return -1
     # Menge der Lösungsindizes muss = der Menge der Indizes von 0 bis n-1 sein
    if set(permutation) != set(range(n)):
        print ("Not a proper permutation!")
        return  -1
    
    total_distance = 0
    for i in range(n):
        if i < n-1:
            total_distance += distance_matrix[permutation[i],permutation[i+1]]
        else:
            total_distance += distance_matrix[permutation[i],permutation[0]]   
            
    return total_distance    

In [4]:
def print_obj_and_eval_tsp_solution(distance_matrix, tour, distance):
    
    eval_distance = evaluate_tsp_solution(distance_matrix, tour)
    
    if distance == eval_distance:
        print ("Solution feasible, distance is: ", distance)
    elif eval_distance < 0:
        print("Solution infeasible")
    else: 
        print("Solution feasible, wrong distance: ", distance, " evaluation gave ", eval_distance)

## Nearest Neighbor als Greedy-Verfahren für das TSP: Hilfsfunktion


In [5]:
@njit
def get_dist_feasible_candidate_tours(tour, total_distance, distance_matrix):
    
    dist_candidate_tour = []
    for neighbor, distance in enumerate(distance_matrix[tour[-1]]):
        if neighbor in tour:
            continue
        
        candidate_tour = tour + [neighbor]
        
        if len(candidate_tour) == len(distance_matrix):
            distance += distance_matrix[neighbor, tour[0]]
        
        total_distance_candidate_tour = total_distance + distance
        dist_candidate_tour.append((total_distance_candidate_tour, candidate_tour))
        
        
    return dist_candidate_tour


In [6]:
@njit
def tsp_greedy(distance_matrix, tour):
        
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(tour) < len(distance_matrix):    
        
        #bestimme alle zulässigen Nachbarn und deren Entfernungen
        dist_candidate_tours = get_dist_feasible_candidate_tours(tour, total_distance, distance_matrix)   
        
        # bestimme das minimale Tupel (der erste Wert des Tupels, d.h. die Distance, wird automatisch genutzt)
        distance_best_candidate_tour, best_candidate_tour = min(dist_candidate_tours)
        
        tour = best_candidate_tour
        total_distance = distance_best_candidate_tour    
        

    return tour, total_distance

## Nearest Neighbor als Greedy-Verfahren für das TSP: Hauptfunktion

- die Hauptfunktion `tsp_greedy` sieht dann auch leicht anders aus:
  - beachte: der erste Parameter ist eine Tour!
  - und: durch die Verwendung des tupels `(distance, tour)` für jeden Nachbarn können wir einfach das Minimum aus der Liste der Tupel suchen (denn das erste Element wird zum Vergleich zuerst herangezogen)

## Erste Verbesserungsidee: Multi-Start-Greedy

- das Ergebnis der Greedy-Heuristik ist offenbar nicht sehr gut
- **aber:** die Greedy-Heuristik ist schnell - es ist nicht teuer, sie aufzurufen
  - dies kann man nutzen, um verbesserte Heuristiken "um Greedy-Vefahren herum" zu bauen

**Erste einfache Idee: Starte Greedy mit verschiedenen (allen) Startknoten** 
- die Nearest-Neighbor-Heuristik hat für unterschiedliche Startpunkte unterschiedliche Werte



In [7]:
def tsp_greedy_multi_start(distance_matrix):
    best_distance = 999999
    best_tour = None
    for start_node in range(len(distance_matrix)):
        tour, distance = tsp_greedy(distance_matrix, [start_node])
        if distance < best_distance:
            best_distance = distance
            best_tour = tour         

    return best_tour, best_distance  

# Beam Search: Einschränkung der Breitensuche

- Idee: ein Kompromiss zwischen Greedy und Breitensuche
- übernimm auf jeder Stufe nur eine bestimmte Anzahl (beam-width) von Zustandsknoten
  - Auswahl z.B. auf Basis von Distanz der Tour
- hier einfache Implementierung mit der Funktion `nsmallest` aus dem Paket `heapq`

In [8]:
from heapq import nsmallest
def tsp_beam_search(distance_matrix, tour,  beam_width):
        
    total_distance = 0
    
    
    # alle Zustandsknoten als Distanz-Tour-Tupel in der aktuellen Stufe / Ebene 
    dist_tours_current_stage = [(0, tour.copy())] 
    
    for stage in range(0,len(distance_matrix)-1):
        
        dist_tours_next_stage = []
        
        for tour_dist, current_tour in dist_tours_current_stage:
                  
            #bestimme alle zulässigen Tourerweiterungen und deren Entfernungen
            dist_candidate_tours = get_dist_feasible_candidate_tours(current_tour, tour_dist, distance_matrix)     
            
        
            for candidate_tour_dist, candidate_tour in dist_candidate_tours:                         

                dist_tours_next_stage.append( (candidate_tour_dist, candidate_tour) )
                
        dist_tours_current_stage = nsmallest(beam_width, dist_tours_next_stage)
    
    best_distance, best_tour  = min(dist_tours_current_stage)
    return best_tour, best_distance

## Greedy mit Rollout als Beispiel für die Nutzung einer heuristischen Funktion (für das TSP)

- wir haben bereits eine einfaches Beispiel für eine heuristische Funktion kennengelernt:
- **Rollout**, wobei eine Greedy-Algroithmus von jedem Kandidatenzustand bis zum Ende ausgeführt wird, um eine **Abschätzung** des zukünftigen Wertes jedes Kandidaten zu erhalten!

- die folgende Funktion erhält einen Zustand (d.h. eine Teil-Tour) und eine Distanzmatrix und gibt die Kosten den Greedy bi zum Ende zurück (beachte: die "bisherigen" Kosten fließen nicht ein!)

In [9]:
@njit # Abschätzung mittels "Ausrollen" des Greedy vom "jetzigen" Zustand
def heuristic_rollout(distance_matrix, tour):
    _, distance = tsp_greedy(distance_matrix, tour)
    return distance

## Greedy mit einer heuristischen Funktion zur Auswahl (Rollout)

- wir stellen nun eine allgemeine Implementierung für die Verwendung einer heuristischen Funktion vor

- der letzte Parameter `heuristic function` ist eine **Funktion**, d.h. wir können eine beliebige Funktion nutzen
  - wir werden dann die Funktion `heuristic_rollout` übergeben

In [10]:
@njit
def tsp_greedy_with_heuristic(distance_matrix, tour, heuristic_function = heuristic_rollout):
    
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(tour) < len(distance_matrix):  
        
        #bestimme für alle zulässigen Nachbarn die Teil-Touren deren Entfernungen
        dist_candidate_tours = get_dist_feasible_candidate_tours(tour, total_distance, distance_matrix)   
        
        # berechne für alle Kandidaten-Touren mit Hilfe der Heuristik den Wert f(n) = g(n)+h(n)
        # estimates_candidates_tours gibt uns für jede Kandidatentour ein Triple (f(n), g(n), tour), d.h. (estimate, dist, tour)    
        estimates_candidate_tours = get_estimates_candidate_tours(dist_candidate_tours, distance_matrix, heuristic_function)
    
        # bestimme das minimale Tupel (der erste Wert des Tupels, d.h. die Abschätzung f(x), wird automatisch genutzt)
        estimate_best_candidate_tour, dist_best_candidate_tour, best_candidate_tour = min(estimates_candidate_tours)
        
        tour = best_candidate_tour
        total_distance = dist_best_candidate_tour # wichtig: Hier nicht das estimate, sondern die Distanz (g(n))   
        
        
    return tour, total_distance

## Die Hilfsfunktion `get_estimates_candidate_tours`

..berechnet mit Hilfe einer heuristischen Funktion `heuristic_function` ($h(n)$) für jede Kandidatentour $n$ die geschätzte Gesamtlänge $f(n) = g(n) + h(n)$ 

Parameter: 
- Liste mit Tupeln `(dist, tour`) für alle Kandidaten
- Distanzmatrix 
- eine heuristische Funktion `heuristic_function`

Rückgabe:
- Liste mit 3-Tupeln (triples) `(estimate, dist, tour)` für jede Kandidatentour


In [11]:
@njit
def get_estimates_candidate_tours(dist_candidate_tours, distance_matrix, heuristic_function):
    
    estimates_candidate_tours = []
    for dist, candidate_tour in dist_candidate_tours:
        heuristic_value = heuristic_function(distance_matrix, candidate_tour)
        estimate = dist + heuristic_value
        estimates_candidate_tours.append((estimate, dist, candidate_tour))
        
    return estimates_candidate_tours

## Nun haben wir alles beisammen für ein "informiertes Greedy" / Greedy mit Heuristischer Funktion:

..probieren wir es aus!

In [48]:
tour, distance = tsp_greedy_with_heuristic(distance_matrix, [0])

print (distance)

8042


In [13]:
@njit
def tsp_informed_beam_search(distance_matrix, tour, beam_width, heuristic_function):
        
    total_distance = 0
    
    # alle Zustandsknoten als Estimate-Distanz-Tour-Tupel in der aktuellen Stufe / Ebene 
    estimate_tours_current_stage = [(0, 0, tour)] 
    
    for stage in range(0,len(distance_matrix)-1):
        
        estimate_tours_next_stage = []
        
        for tour_estimate, tour_dist, current_tour in estimate_tours_current_stage:
                  
            #bestimme alle zulässigen Tourerweiterungen und deren Entfernungen
            dist_candidate_tours = get_dist_feasible_candidate_tours(current_tour, tour_dist, distance_matrix)  
            
            # berechne für alle Kandidaten-Touren mit Hilfe der Heuristik den Wert f(n) = g(n)+h(n)
            # estimates_candidates_tours gibt uns für jede Kandidatentour ein Triple (f(n), g(n), tour), d.h. (estimate, dist, tour)    
            estimates_candidate_tours = get_estimates_candidate_tours(dist_candidate_tours, distance_matrix, heuristic_function)
        
            for estimate_candidate_tour, dist_candidate_tour, candidate_tour in estimates_candidate_tours:                         

                estimate_tours_next_stage.append( (estimate_candidate_tour, dist_candidate_tour, candidate_tour) )
        
        # wähle die besten aus zur Betrachtung in der nächsten Stufe!
        estimate_tours_current_stage = nsmallest(beam_width, estimate_tours_next_stage)
        #print (stage, estimate_tours_current_stage)
        
    best_estimate, best_distance, best_tour  = min(estimate_tours_current_stage)
    return best_tour, best_distance

## Informierte Suche mit begrenzter Anwendung der Heuristik

#### Beobachtung:

- wenn man in jedem Suchknoten einmal einen kompletten Greedy **für jeden Nachbarn / Kandidaten** ausführt, dann ist das sehr aufwändig!
- insbesondere wird auch für "Nachbarn", die sehr weit weg liegen, trotzdem der Rollout durchgführt

#### Idee: Limitiere die Anwendung der Heuristik auf "vielversprechende Fälle"
- wir führen einen Parameter `no_candidates_for_heuristic` ein, der besagt, wie viele Nachbarn maximal mittels Heuristik betrachtet werden solln
- und betrachten nur die `no_candidates_for_heuristic` Nachbarn / Kandidaten, die nach dem Kriterium $g(n)$ (hier: Distanz der Teiltour bis zum Kandidaten) am besten sind:

#### Anmerkung:

Bertsekas nennt diese Idee im Kontext von Rollout  **simplified rollout** (siehe A+D-Projekt vom letzten Jahr)


In [14]:
@njit
def get_estimates_candidate_tours_limited(dist_candidate_tours, distance_matrix, heuristic_function, no_candidates_for_heuristic):
    
    estimates_candidate_tours = []
    
    # hier erfolgt die Auswahl der zu betrachtenden Kandidaten anhand der Distanz 
    # beachte: gleiche logik mit nsmallest wie schon oben
    dist_candidate_tours_limited = nsmallest(no_candidates_for_heuristic, dist_candidate_tours )
    
    for dist, candidate_tour in dist_candidate_tours_limited:
        heuristic_value = heuristic_function(distance_matrix, candidate_tour)
        estimate = dist + heuristic_value
        estimates_candidate_tours.append((estimate, dist, candidate_tour))
        
    return estimates_candidate_tours

## Begrenzte Anwendung der Heuristik bei Greedy

In [15]:
@njit
def tsp_greedy_with_heuristic_limited(distance_matrix, tour, heuristic_function, no_candidates_for_heuristic):
    
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(tour) < len(distance_matrix):    
        
        #bestimme alle zulässigen Nachbarn und deren Entfernungen
        dist_candidate_tours = get_dist_feasible_candidate_tours(tour, total_distance, distance_matrix)   
        
        
        estimates_candidate_tours = get_estimates_candidate_tours_limited(dist_candidate_tours, distance_matrix, heuristic_function, no_candidates_for_heuristic)
    
        # bestimme das minimale Tupel (der erste Wert des Tupels, d.h. die Distance, wird automatisch genutzt)
        estimate_best_candidate_tour, dist_best_candidate_tour, best_candidate_tour = min(estimates_candidate_tours)
        
        tour = best_candidate_tour
        total_distance = dist_best_candidate_tour    
        
        
    return tour, total_distance

..Test:

In [47]:
tsp_greedy_with_heuristic_limited(distance_matrix, [1], heuristic_rollout, 10)[1]

7970

## Begrenzte Anwendung der Heuristik in Beam Search

In [17]:
@njit
def tsp_informed_beam_search_limited(distance_matrix, tour, beam_width, heuristic_function, no_candidates_for_heuristic):
        
    total_distance = 0
    
    # alle Zustandsknoten als Estimate-Distanz-Tour-Tupel in der aktuellen Stufe / Ebene 
    estimate_tours_current_stage = [(0, 0, tour)] 
    
    for stage in range(0,len(distance_matrix)-1):
        
        estimate_tours_next_stage = []
        
        for tour_estimate, tour_dist, current_tour in estimate_tours_current_stage:
                  
            #bestimme alle zulässigen Tourerweiterungen und deren Entfernungen
            dist_candidate_tours = get_dist_feasible_candidate_tours(current_tour, tour_dist, distance_matrix)  

            estimates_candidate_tours = get_estimates_candidate_tours_limited(dist_candidate_tours, distance_matrix, heuristic_function, no_candidates_for_heuristic)
            
            for estimate_candidate_tour, dist_candidate_tour, candidate_tour in estimates_candidate_tours:                  
                estimate_tours_next_stage.append( (estimate_candidate_tour, dist_candidate_tour, candidate_tour) )
              
        estimate_tours_current_stage = nsmallest(beam_width, estimate_tours_next_stage)

    best_estimate, best_distance, best_tour  = min(estimate_tours_current_stage)
    return best_tour, best_distance

# Experimente und deren Auswertung

## Herausforderungen für die Auswertung

**Wir wollen:**
- nicht nur eine, sondern mehrere Instanzen testen
  - ohne jedes Mal jede Zelle des Notebooks laufen zu lassen
- die Ergebnisse in Dateien speichern
  - Läufe können länger dauern und an verschiedenen Tagen passieren
  - die Auswertung kann später gemacht wercen
- damit umgehen, dass es nicht nur verschiedene Algorithmen, sondern auch Varianten und paramtrisierte Algorithmen gibt
  - z.B. Anzahl betrachteter Knote / Zustände (`beam width`) bei Beam Search, Limit für die Anwendung von Heuristiken in der informierten Suche
- sinnvolle Tabellen erstellen
  - Ergebnisse und Laufzeiten
  - Zusammenfassung / Normierung von Ergebnissen mittes Abstand zur optimalen Lösung


## Mehrere Algorithmen laufen lassen

- wir schreiben eine Funktion, die 
  - für eine Instanz mehrere Algorithmen aufruft
  - und die Ergebnisse über print-statements ausgibt
  

In [21]:
from timeit import default_timer as timer

def run_algorithms(instance_name):
    
    tsplib_file = "./../problems/tsp/instances/" + instance_name + ".tsp" # optimale Lösung 7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
    distance_matrix = tsplib_distance_matrix(tsplib_file)
    print ("Instanz", instance_name)
    print ("Nodes", len(distance_matrix))
    
    print("Nearest Neighbor:")
    starttime = timer()    
    permutation_nn, distance_nn = tsp_greedy(distance_matrix, [0])
    print_obj_and_eval_tsp_solution(distance_matrix, permutation_nn, distance_nn )
    print(f"Time: {timer()- starttime:0.5f}")
    
    print("Mutltistart Nearest Neighbor:")
    starttime = timer()    
    permutation_nn, distance_nn = tsp_greedy_multi_start(distance_matrix)
    print_obj_and_eval_tsp_solution(distance_matrix, permutation_nn, distance_nn )
    print(f"Time: {timer()- starttime:0.5f}")
    
    print("Beam Search Width 20")
    starttime = timer()    
    permutation_nn, distance_nn  = tsp_beam_search(distance_matrix, [0], 20)
    print_obj_and_eval_tsp_solution(distance_matrix, permutation_nn, distance_nn )
    print(f"Time: {timer()- starttime:0.3f}")
     
    
    print("Informed Beam Search Width 20")
    starttime = timer()    
    permutation_nn, distance_nn  = tsp_informed_beam_search(distance_matrix, [0], 20, heuristic_rollout)
    print_obj_and_eval_tsp_solution(distance_matrix, permutation_nn, distance_nn )
    print(f"Time: {timer()- starttime:0.3f}")
    
    
    print("Informed Beam Search Width 20 Limited HeurEvals 10")
    starttime = timer()    
    permutation_nn, distance_nn  = tsp_informed_beam_search_limited(distance_matrix, [0], 20, heuristic_rollout,10)
    print_obj_and_eval_tsp_solution(distance_matrix, permutation_nn, distance_nn )
    print(f"Time: {timer()- starttime:0.3f}")
    

In [23]:
run_algorithms("berlin52")

Instanz berlin52
Nodes 52
Nearest Neighbor:
Solution feasible, distance is:  8980
Time: 0.00087
Mutltistart Nearest Neighbor:
Solution feasible, distance is:  8181
Time: 0.03850
Beam Search Width 20
Solution feasible, distance is:  10129
Time: 0.084
Informed Beam Search Width 20
Solution feasible, distance is:  8002
Time: 7.728
Informed Beam Search Width 20 Limited HeurEvals 10
Solution feasible, distance is:  7997
Time: 2.092


## Eine Funktion zum Experimentieren mit mehreren Instanzen
- wir möchten nicht  nur eine, sondern mehrere Instanzen testen

In [25]:
def run_algorithms_on_instances(instance_names):

    for instance_name in instance_names: 
        run_algorithms(instance_name)
        

In [27]:
run_algorithms_on_instances(["berlin52", "gr48"])  

Instanz berlin52
Nodes 52
Nearest Neighbor:
Solution feasible, distance is:  8980
Time: 0.00077
Mutltistart Nearest Neighbor:
Solution feasible, distance is:  8181
Time: 0.03346
Beam Search Width 20
Solution feasible, distance is:  10129
Time: 0.088
Informed Beam Search Width 20
Solution feasible, distance is:  8002
Time: 8.009
Informed Beam Search Width 20 Limited HeurEvals 10
Solution feasible, distance is:  7997
Time: 1.959
Instanz gr48
Nodes 48
Nearest Neighbor:
Solution feasible, distance is:  6098
Time: 0.00060
Mutltistart Nearest Neighbor:
Solution feasible, distance is:  5840
Time: 0.02334
Beam Search Width 20
Solution feasible, distance is:  6372
Time: 0.065
Informed Beam Search Width 20
Solution feasible, distance is:  5464
Time: 5.457
Informed Beam Search Width 20 Limited HeurEvals 10
Solution feasible, distance is:  5482
Time: 1.594


## Daten strukturiert erfassen

- im obigen Code wurden die Daten relativ unstrukturiert ausgegeben
- man kann dann die Daten zwar per copy/paste "herausschreiben", das ist jedoch relativ aufwändig und umständlich

**Neuer Ansatz:**
- wir sammeln die Daten für jeden Lauf eines Algorithmus in einem `dict` mit den folgenden Daten
  - Instance
  - Algorithm
  - verwendete Parameter
  - Objective
  - Time
- fügen all diese Ergebnisse zusammen
- und bilden daraus einen `dataframe`
  
 

## Daten strukturiert erfassen: Funktion, die einen Algorithmus ausführt und ein `dict` zurückgibt

Die folgende Funktion `run_algorithm_with_parameters` hat folgende Argumente:
- den Namen der Instanz
- die Funktion, die ausgeführt werden soll 
- ein `dict` `algorithm_parameters`mit den Parametern der Funktion (alle außer der Distanzmatrix) 
  - Beispiel: `{ "tour" : [0] }` für den Greedy-Algorithmus wird der Funktionsparameter `tour` (die zunächst nur den Start-Knoten enthält), als `[0]` festgelegt
  - wichtig: Parameter-Name ist ein string!

Auf diese Weise können **unterschiedliche Algorithmen**, die insbesondere eine  **unterschiedliche Anzahl an Parametern haben können**, einheitlich aufgerufen werden
 

Die Funktion führt den Algorithmus aus
- der Ausdruck `**algorithm_parameters` beim Aufruf der Funktion "entpackt" das dict so, dass die Elemente des `dict` als Funktionsparameter genutzt werden
- wichtig: die `keys` des `dict` müssen genau so heißen wie die Parameter der Algorithmen-Funktion!

Schließlich gibt die Funktion ein `dict` zurück mit den wichtigsten Ergebnissen

In [28]:
def run_algorithm_with_parameters(instance_name, function_algorithm, algorithm_parameters):
    
    ## hier ggf den Dateipfad anpassen
    tsplib_file = "./../problems/tsp/instances/" + instance_name +".tsp" # 
    distance_matrix = tsplib_distance_matrix(tsplib_file)  

    # ausführen
    starttime = timer()    
    permutation, distance = function_algorithm(distance_matrix, **algorithm_parameters)
    
    result_dict = {}
    result_dict["Instance"] = instance_name
    result_dict["Algorithm"] = function_algorithm.__name__
    result_dict["Parameters"] = algorithm_parameters
    result_dict["Objective"] = distance
    result_dict["Time"] = timer()- starttime
    
    return result_dict
    

Beispiele:

In [29]:
# Beispiel: der greedy nearest neighbor hat einen Parameter (tour, d.h. start-Knoten)
parameters = { "tour" : [0]}
result = run_algorithm_with_parameters("berlin52", tsp_greedy, parameters)
print(result)

# Beispiel: beam search hat zwei Parameter
parameters = { "tour" : [0], "beam_width" :500 }
result = run_algorithm_with_parameters("berlin52", tsp_beam_search, parameters)
print(result)


{'Instance': 'berlin52', 'Algorithm': 'tsp_greedy', 'Parameters': {'tour': [0]}, 'Objective': 8980, 'Time': 0.000720599999993965}
{'Instance': 'berlin52', 'Algorithm': 'tsp_beam_search', 'Parameters': {'tour': [0], 'beam_width': 500}, 'Objective': 8446, 'Time': 3.9379715000000033}


## Mehrere Algorithmen auswerten und Ergebnisse als Data Frame darstellen

- nun haben wir eine einheitliche Funktion, die Daten strukturiert ausgibt
- wir schreiben nun eine Funktion, die mehrere Algorithmen ausführt
- diese Funktion hat als Parameter:
  - einen Instanznamen
  - eine Liste aus 2- `tuples`, bestehend aus: `algorithmus_funktion` und einem `dict` mit Paramtern
  
..die Funktion führt die Algorithmen aus und gibt eine Liste mit Ergebnis-`dict`s zurück


In [30]:
def run_algorithms_with_parameters(instance_name, algorithms_parameters):
    
    results = []
    for algorithm, parameters in algorithms_parameters:
    
        results.append (run_algorithm_with_parameters(instance_name, algorithm, parameters) )
    
    
    
    return results

In [31]:
algorithms_parameters = [] # Liste erstellen

# greedy (beachte: jeder Eintrag der Liste ist ein Tupel (algorithmus, parameter-dict )  )
algorithms_parameters.append( ( tsp_greedy, { "tour" : [0] } ) ) 

algorithms_parameters.append( ( tsp_beam_search, { "tour" : [0],
                                                  "beam_width": 500 } ) )

results = run_algorithms_with_parameters( "berlin52", algorithms_parameters) 

print(results)

[{'Instance': 'berlin52', 'Algorithm': 'tsp_greedy', 'Parameters': {'tour': [0]}, 'Objective': 8980, 'Time': 0.0010986000000059448}, {'Instance': 'berlin52', 'Algorithm': 'tsp_beam_search', 'Parameters': {'tour': [0], 'beam_width': 500}, 'Objective': 8446, 'Time': 4.042473300000012}]


## Ergebnisse als Data Frame darstellen

..der nächste Schritt ist dann relativ einfach:

- eine Liste von `dict`s kann einfach in ein data frame überführt werden

In [32]:
pd.DataFrame(results)

,Instance,Algorithm,Parameters,Objective,Time
0,berlin52,tsp_greedy,{'tour': [0]},8980,0.001099
1,berlin52,tsp_beam_search,"{'tour': [0], 'beam_width': 500}",8446,4.042473


## Erweitern der Ergebnisse 

.. wir können auch die Results um weitere Läufe ergänzen:

In [33]:

algorithms_parameters = []


algorithms_parameters.append(( tsp_greedy_multi_start, {} ))

algorithms_parameters.append(( tsp_informed_beam_search, {  "tour" : [0],
                                                            "beam_width": 20, 
                                                            "heuristic_function" : heuristic_rollout } ))

algorithms_parameters.append(( tsp_informed_beam_search_limited, { "tour" : [0], 
                                                                   "beam_width": 20, 
                                                                   "heuristic_function" : heuristic_rollout,
                                                                   "no_candidates_for_heuristic" : 10 } ))


results += run_algorithms_with_parameters( "berlin52", algorithms_parameters) 


pd.DataFrame(results)   

,Instance,Algorithm,Parameters,Objective,Time
0,berlin52,tsp_greedy,{'tour': [0]},8980,0.001099
1,berlin52,tsp_beam_search,"{'tour': [0], 'beam_width': 500}",8446,4.042473
2,berlin52,tsp_greedy_multi_start,{},8181,0.034619
3,berlin52,tsp_informed_beam_search,"{'tour': [0], 'beam_width': 20, 'heuristic_fun...",8002,7.882782
4,berlin52,tsp_informed_beam_search_limited,"{'tour': [0], 'beam_width': 20, 'heuristic_fun...",7997,2.051019



><div class="alert alert-block alert-info">
<b>
Ergänzen Sie die Funktion `run_algorithms` so, dass der Gap zur optimalen Lösung als Ergebniswert ausgegeben wird!
</b></div>  

    

## Mehrere Instanzen vergleichen

Nun können wir relativ einfach eine neue Funktion schreiben, die die Experimente für mehrere Instanzen ausführt:



In [34]:
def run_algorithms_with_parameters_on_instances (instance_names, algorithms_parameters):
    
    results = []
    
    for instance_name in instance_names: 
        results += run_algorithms_with_parameters( instance_name, algorithms_parameters) 
        
    return results


..probieren wir es aus:


In [35]:
algorithms_parameters = [] # Liste erstellen

# greedy (beachte: jeder Eintrag der Liste ist ein Tupel (algorithmus, parameter-dict )  )
algorithms_parameters.append( ( tsp_greedy, { "tour" : [0] } ) ) 

algorithms_parameters.append( ( tsp_beam_search, { "tour" : [0],
                                                  "beam_width": 500 } ) )

instances = ["berlin52", "gr48"]

results = run_algorithms_with_parameters_on_instances (instances, algorithms_parameters)

df_results = pd.DataFrame(results)   
df_results

,Instance,Algorithm,Parameters,Objective,Time
0,berlin52,tsp_greedy,{'tour': [0]},8980,0.000799
1,berlin52,tsp_beam_search,"{'tour': [0], 'beam_width': 500}",8446,3.961152
2,gr48,tsp_greedy,{'tour': [0]},6098,0.000580
3,gr48,tsp_beam_search,"{'tour': [0], 'beam_width': 500}",6315,3.388942


## Ergebnisse aufbereiten mit Pivot-Tabellen

- die Ergebnistabelle von oben ist etwas redundant 
- was, wenn wir z.B. nur eine Zeile je Instanz haben wollen und für jeden Algorithmus eine Spalte?

- derartige Umstellungen können wir mit so genannten Pivot-Tabellen machen:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html



In [36]:
df_pivot = df_results.pivot_table(index="Instance", columns="Algorithm", values=["Objective"])
df_pivot

Objective           
Algorithm tsp_beam_search tsp_greedy
Instance                            
berlin52             8446       8980
gr48                 6315       6098

...wir können auch zwei Arten von Ergebnissen nebeneinander Anzeigen:

In [37]:
df_pivot = df_results.pivot_table(index="Instance", columns="Algorithm", values=["Objective", "Time"])
df_pivot

Objective                       Time           
Algorithm tsp_beam_search tsp_greedy tsp_beam_search tsp_greedy
Instance                                                       
berlin52             8446       8980        3.961152   0.000799
gr48                 6315       6098        3.388942   0.000580

..oder die Mittelwerte der Zeilen und Spalten mit anzeigen lassen

In [38]:
df_pivot = df_results.pivot_table(index="Instance", columns="Algorithm", values=["Time"], margins=True)
df_pivot

Time                     
Algorithm tsp_beam_search tsp_greedy       All
Instance                                      
berlin52         3.961152   0.000799  1.980975
gr48             3.388942   0.000580  1.694761
All              3.675047   0.000690  1.837868

## Ausgabe in LaTeX-Tabelle
- falls Sie LaTeX nutzen, können Sie direkt LaTeX-Code zur Darstellung der Tabelle erzeugen:

In [39]:
print(df_pivot.to_latex())

\begin{tabular}{lrrr}
\toprule
{} & \multicolumn{3}{l}{Time} \\
Algorithm & tsp\_beam\_search & tsp\_greedy &       All \\
Instance &                 &            &           \\
\midrule
berlin52 &        3.961152 &   0.000799 &  1.980975 \\
gr48     &        3.388942 &   0.000580 &  1.694761 \\
All      &        3.675047 &   0.000690 &  1.837868 \\
\bottomrule
\end{tabular}



C:\Users\mroemer\AppData\Local\Temp\ipykernel_21052\3636991984.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_pivot.to_latex())


## Schreiben in und Lesen aus Dateien

- oftmals können nicht alle Experimente und Auswertungen am Stück durchgeführt werden
- weiterhin möchte man häufig später weitere Analysen machen und die Auswertungen auch später noch nachvollziehen können
- mit dataframes geht das sehr leicht:

- schreiben des (ursprünglichen) data frames in eine Datei:

In [40]:
file_name = "results.txt"
df_results.to_csv(file_name, index=False) # csv heisst comma-separated values

- einlesen in ein Data Frame:

## Beispiel für ein einheitliches Qualitätsmaß: Der Gap zur optimalen Lösung

- wenn wir Instanzen mit unterschiedlich skalierten Zielfunktionswerten (Distanzen) haben, sind Ergebnisse nur schwer einheitlich zu vergleichen bzw. aggregiert zu betrachten
- besser ist es daher oft, die relative Performance eines Algorithmus zu einer Bezugsgröße zu betrachten, z.B.:
  - eine optimale Lösung
  - die Lösung eines anderen Algorithmus

Da wir die optimalen Lösungen kennen, können wir diese in einem dict speichern:

In [41]:
instance_optimal_value = {"gr48":5046, "brazil58":25395, "berlin52":7542}

..und dann berechen wir den relativen Gap (den relativen Abstand zur optimalen Lösung):
- wir nutzen dazu den optimalen Wert als Grundlage, durch den wir die Differenz aus der gefundenen und der optimalen Lösung teilen
- wenn wir die optimale Lösung finden, ist der Gap 0 %

In [42]:
def get_opt_gap(objective_value, instance_name):
    return (objective_value - instance_optimal_value[instance_name]) / instance_optimal_value[instance_name]


Beispiel:


In [43]:
parameters = { "tour" : [0]}
result = run_algorithm_with_parameters("berlin52", tsp_greedy, parameters)
print("Greedy ", result["Objective"])

print("Optimal ",  instance_optimal_value["berlin52"])

print("Gap ", get_opt_gap(result["Objective"], "berlin52"))

Greedy  8980
Optimal  7542
Gap  0.19066560594006896


## Einbau des Optimality Gap in die Auswertungs-Funktion:




In [44]:
def run_algorithm_with_parameters(instance_name, function_algorithm, algorithm_parameters):
    
    ## hier ggf den Dateipfad anpassen
    tsplib_file = "./../problems/tsp/instances/" + instance_name +".tsp" # 
    distance_matrix = tsplib_distance_matrix(tsplib_file)  

    # ausführen
    starttime = timer()    
    permutation, distance = function_algorithm(distance_matrix, **algorithm_parameters)
    
    result_dict = {}
    result_dict["Instance"] = instance_name
    result_dict["Algorithm"] = function_algorithm.__name__
    result_dict["Parameters"] = algorithm_parameters
    result_dict["Objective"] = distance
    result_dict["OptGap"] = get_opt_gap(distance, instance_name)
    result_dict["Time"] = timer()- starttime
    
    return result_dict
    

## Untersuchen der Auswirkung von Parametern 

- viele komplexere Algorithmen haben **Parameter**, die ihr Verhalten beeinflussen, z.B.
   - Startort beim Nearest Neighbor
   - Breite (beam_width) beim Beam Search
   - Anzahl an betrachteten Nachbarn bei limitierter Anwendung informierter Suche
- oftmals ist es interessant, die Auswirkung der wesentlichen Parameter auf Ergebnis und Laufzeit zu untersuchen!



## Anpassung der Run Algorithm-Funktion

- wir wollen nun, dass für alle Parameter ein eigener Wert angelegt wird
- wir passsen die Funktion `run_algorithm_with_parameters` entsprechend an:

In [45]:
def run_algorithm_with_parameters(instance_name, function_algorithm, algorithm_parameters):
    
    ## hier ggf den Dateipfad anpassen
    tsplib_file = "./../problems/tsp/instances/" + instance_name +".tsp" # 
    distance_matrix = tsplib_distance_matrix(tsplib_file)  

    # ausführen
    starttime = timer()    
    permutation, distance = function_algorithm(distance_matrix, **algorithm_parameters)
    
    result_dict = {}
    result_dict["Instance"] = instance_name
    result_dict["Algorithm"] = function_algorithm.__name__
    
    ## hier wird ein einzelnes Feld für jeden Parameter angelegt ausser für den Tour-Parameter:
    for name, value in algorithm_parameters.items():
        if name != "tour":
             result_dict[name] = value        
   
    result_dict["Objective"] = distance
    result_dict["OptGap"] = get_opt_gap(distance, instance_name)
    result_dict["Time"] = timer()- starttime
    
    return result_dict
    

.. nun können wir das z.B. für Beam Search ausprobieren:

In [46]:
results = []
algorithm_parameters = {"tour" : [0], "beam_width" : 1}

for beam_width in [1,10,50,100,500]:
    algorithm_parameters["beam_width"] = beam_width
    results.append(run_algorithm_with_parameters("berlin52", tsp_beam_search, algorithm_parameters))
    

pd.DataFrame(results) 
    

,Instance,Algorithm,beam_width,Objective,OptGap,Time
0,berlin52,tsp_beam_search,1,8980,0.190666,0.006351
1,berlin52,tsp_beam_search,10,9423,0.249403,0.051271
2,berlin52,tsp_beam_search,50,9497,0.259215,0.262090
3,berlin52,tsp_beam_search,100,9846,0.305489,0.722041
4,berlin52,tsp_beam_search,500,8446,0.119862,4.097409


## Zusammenfassung

Heute haben wir einige Techniken zum Experimentieren kennengelernt:
- Aufrufen mehrerer Instanzen und Algorithmen
- Datenstrukturen für Ergebnisse
- Aufbereiten der Ergebnisse mit data frames
- Speichern von Ergebnissen in Dateien
- Auslesen von Ergebnissen aus Dateien


**Bitte beachten Sie:**
- Sie können natürlich auch anders arbeiten, z.B. direkt in Dateien schreiben
- Hier haben wir nur zwei Instanzen genutzt - es wäre schön, wenn Sie (deutlich) mehr Instanzen für Ihre Experimente nutzen würden!